In [1]:
import pandas as pd
import json
import ccxt
from algos.daddy.live_trader import liveTrading


In [2]:
symbol='ETH'
TESTNET=False
config_file='algos/eth_daddy/exchanges.csv'
parameter_file="algos/eth_daddy/parameters.json"

In [3]:
EXCHANGES = pd.read_csv(config_file)

In [4]:
lts = {}

In [5]:
for idx, details in EXCHANGES.iterrows():
    exchange_name = details['exchange']
    name = details['name']

    if details['trade'] == 1:       
        lts[name] = liveTrading(exchange_name, name, symbol=details['ccxt_symbol'],testnet=TESTNET, parameter_file=parameter_file, config_file=config_file) 
        lts[name].set_position()

In [7]:
lt = lts[name]

In [11]:
lt.get_balance()

4.38128215

In [10]:
lt.fill_order('buy', method='10sec_average')

Time at filling order is: 2021-05-03 10:02:08.460758
Sending market buy order for ETH-PERP of size 0.002 on ftx in 2021-05-03 10:02:10.750908
Sending market buy order for ETH-PERP of size 0.002 on ftx in 2021-05-03 10:02:21.317227


KeyboardInterrupt: 

In [10]:
exchange = exchange_name

In [11]:
exchange

'ftx'

In [12]:
name

'eth_ftx'

In [13]:
parameters = json.load(open(parameter_file))

In [14]:
parameters

{'mult': 20.0,
 'percentage_large': 3.0,
 'buy_percentage_large': 5.0,
 'macd': -30.0,
 'rsi': 800.0,
 'previous_days': 3.0,
 'position_since': 5.0,
 'position_since_diff': 0.0,
 'change': 0.2,
 'pnl_percentage': -500.0,
 'close_percentage': 5.0,
 'profit_macd': -11200.0,
 'stop_percentage': 0.5,
 'profit_cap': 20.0,
 'name': 'dont_trade'}